# SCAN 

## Task 

Each example in the SCAN dataset is aimed at converting a natural language command to a sequence of actions. 

$$ InputCommand \longrightarrow OutputSequence$$

## Phrase Structure Grammar 

The input commands can be generated with a basic PS grammar starting from C and ending with U: 

1. C $\longrightarrow$ S and S
2. C $\longrightarrow$ S after S
3. C $\longrightarrow$ S
4. S $\longrightarrow$ V twice
5. S $\longrightarrow$ V thrice
6. S $\longrightarrow$ V
7. V $\longrightarrow$ D[1] opposite D[2]
8. V $\longrightarrow$ D[1] around D[2]
9. V $\longrightarrow$ D
10. V $\longrightarrow$ U
11. D $\longrightarrow$ U left
12. D $\longrightarrow$ U right
13. D $\longrightarrow$ turn left
14. D $\longrightarrow$ turn right
15. U $\longrightarrow$ walk
16. U $\longrightarrow$ run
17. U $\longrightarrow$ jump
18. U $\longrightarrow$ look

Where C=Full Command, S= Sentence Phrase, V= Verb Phrase, D= Direction Phrase, U= Verb

## Compositional Abstraction Modelling

Compositionality refers to the ability of compositional generalization i.e the ability to recognize the abstract underlying data structure, recovering the rules of abstraction and productively applying those rules in new contexts.

In the context of SCAN, a CAM (compositional abstraction model) should recover the phrase structure abstraction of the given input and apply it to parse new input and convert it into action sequences. Given such a form of abstraction where the model recovers phrase structure grammar, there can be two possible abstraction models: 

1. Top Down: The highest nodes like C are resolved and interpreted first
2. Bottom Up: The lowest nodes like U are resolved and interpreted first.

If the model has really been able to understand the compositional abstraction of the data in the form of PSG here, it should follow the sequence of PSG to resolve nodes and that can only be accomplished in one of two ways listed above. 

## Top down CAM 

1. Resolve C: Split sentence based on and/after
2. Resolve S: Identify and interpret twice/thrice
3. Resolve V: Identify and interpret opposite/around
4. Resolve D: Identify and interpret left/right/turn left/turn right
5. Resolve U: Identify and interpret all verbs

Example: Jump thrice and turn left

l0=['jump','thrice','and','turn','left']

C/l1=[['jump','thrice'],['turn','left']]

S/l2=[['jump','x x x'],['turn','left']]

V/l3=[['jump','x x x'],['turn','left']]

D/l4=[['jump','x x x'],['LTURN']]

U/l5=[['JUMP JUMP JUMP'],['LTURN']]

In [4]:
def causal_model_td(command):
    # Step 1: Split the command into lexical items (words)
    l0 = command.split()

    # Resolve C: Split based on 'and' or 'after'
    l1 = []
    sub_command = []
    for item in l0:
        if item == "and":
            l1.append(sub_command)
            sub_command = []
        else:
            sub_command.append(item)
    if sub_command:  # Add the last segment
        l1.append(sub_command)

    # Resolve S: Interpret twice/thrice for repetition as individual elements
    l2 = []
    for sub in l1:
        new_sub = []
        i = 0
        while i < len(sub):
            if sub[i] == "twice" and i > 0:
                new_sub.extend([sub[i - 1]] * 2)  # Add two instances of the previous word
                new_sub.pop(-3)  # Remove the original word before twice
            elif sub[i] == "thrice" and i > 0:
                new_sub.extend([sub[i - 1]] * 3)  # Add three instances of the previous word
                new_sub.pop(-4)  # Remove the original word before thrice
            else:
                new_sub.append(sub[i])
            i += 1
        l2.append(new_sub)

    # Resolve V: Interpret opposite/around and handle direction repeats
    l3 = []
    for sub in l2:
        new_sub = []
        i = 0
        while i < len(sub):
            if sub[i] == "around" and i + 1 < len(sub):
                if sub[i + 1] == "RTURN":
                    new_sub.extend(["RTURN"] * 4)  # Equivalent to turning around
                    i += 2  # Skip "RTURN"
                elif sub[i + 1] == "LTURN":
                    new_sub.extend(["LTURN"] * 4)
                    i += 2  # Skip "LTURN"
            elif sub[i] == "opposite" and i + 1 < len(sub):
                if sub[i + 1] == "RTURN":
                    new_sub.extend(["RTURN"] * 2)  # Opposite turn (180-degree)
                    i += 2  # Skip "RTURN"
                elif sub[i + 1] == "LTURN":
                    new_sub.extend(["LTURN"] * 2)
                    i += 2  # Skip "LTURN"
            else:
                new_sub.append(sub[i])
                i += 1
        l3.append(new_sub)


    # Resolve D: Interpret directions (left/right/turn left/turn right)
    l4 = []
    for sub in l3:
        new_sub = []
        skip_next = False
        for i, item in enumerate(sub):
            if skip_next:
                skip_next = False
                continue
            if item == 'turn' and i + 1 < len(sub):
                if sub[i + 1] == 'right':
                    new_sub.append('RTURN')
                    skip_next = True  # Skip 'right'
                elif sub[i + 1] == 'left':
                    new_sub.append('LTURN')
                    skip_next = True  # Skip 'left'
            elif item == 'right':
                new_sub.append('RTURN')
            elif item == 'left':
                new_sub.append('LTURN')
            else:
                new_sub.append(item)
        l4.append(new_sub)

    # Resolve U: Identify and replace all verbs
    actions = {
        "walk": "WALK",
        "run": "RUN",
        "jump": "JUMP",
        "look": "LOOK"
    }
    l5 = []
    for sub in l4:
        new_sub = []
        for item in sub:
            if item.strip() in actions:
                new_sub.append(actions[item.strip()])
            else:
                new_sub.append(item)
        l5.append(new_sub)

    # Flatten the list into one single list
    final_output = []
    for sublist in l5:
        final_output.extend(sublist)

    # Return the fully resolved command as individual actions
    return final_output

# Example usage:
command = "jump thrice and turn left"
output = causal_model_td(command)
print(output)

[['jump', 'jump', 'jump'], ['turn', 'left']]


RuntimeError: No active exception to reraise

## Bottom up CAM

1. Resolve U: Split sentence into lexical items, identify and interpret all verbs.
2. Resolve D: Identify and interpret left/right/turn left/turn right
3. Resolve V: Identify and interpret opposite/around
4. Resolve S: Identify and interpret twice/thrice
5. Resolve C: Identify and interpret and/after.

Example: Jump thrice and turn left

l0=['jump','thrice','and','turn','left']

U/l1=['JUMP','thrice','and],'turn','left']

D/l2=['JUMP','thrice','and','LTURN']

V/l3=['JUMP','thrice','and','LTURN']

S/l4=['JUMP','JUMP','JUMP','and','LTURN']

C/l5=['JUMP','JUMP','JUMP','LTURN']

In [19]:
def causal_model_bu(command):
    # split the command into individual lexical items (words)
    l0 = command.split()

    # resolve U: Interpret action words
    actions = {
        "walk": "WALK",
        "run": "RUN",
        "jump": "JUMP",
        "look": "LOOK"
    }
    l1 = l0.copy()
    for i, item in enumerate(l0):
        if item in actions:  # if l1 has action words, replace with action command
            l1[i] = actions[item]

    # resolve D: Interpret left/right/turn left/turn right
    l2 = []
    skip_next = False
    for i, item in enumerate(l1):
        if skip_next:
            skip_next = False
            continue

        if item == 'turn' and i + 1 < len(l1):
            if l1[i + 1] == 'right':
                l2.append('RTURN')
                skip_next = True  # Skip the next word ('right')
            elif l1[i + 1] == 'left':
                l2.append('LTURN')
                skip_next = True  # Skip the next word ('left')
        elif item == 'right':
            l2.append('RTURN')
        elif item == 'left':
            l2.append('LTURN')
        else:
            l2.append(item)
   

    # Resolve V: Interpret opposite/around and handle direction repeats
    l3 = []
    i = 0
    while i < len(l2):
        if l2[i] == "around" and i + 1 < len(l2):
            if l2[i + 1] == "RTURN":
                l3.extend(["RTURN"] * 4)  # Equivalent to turning around
                i += 2  # Skip "RTURN"
            elif l2[i + 1] == "LTURN":
                l3.extend(["LTURN"] * 4)
                i += 2  # Skip "LTURN"
        elif l2[i] == "opposite" and i + 1 < len(l2):
            if l2[i + 1] == "RTURN":
                l3.extend(["RTURN"] * 2)  # Equivalent to 180-degree turn
                i += 2  # Skip "RTURN"
            elif l2[i + 1] == "LTURN":
                l3.extend(["LTURN"] * 2)
                i += 2  # Skip "LTURN"
        else:
            l3.append(l2[i])
            i += 1
    

    # Resolve S: Interpret twice/thrice for repetition as individual elements
    l4 = []
    i = 0
    while i < len(l3):
        if l3[i] == "twice" and i > 0:
            l4.extend([l4[-1]] * 1)  # Add one more instance to repeat twice
        elif l3[i] == "thrice" and i > 0:
            l4.extend([l4[-1]] * 2)  # Add two more instances to repeat thrice
        else:
            l4.append(l3[i])
        i += 1
    

    # Resolve C: Handle 'and'/'after' conjunctions
    l5 = []
    if "and" in l4:
        l5 = [item for item in l4 if item != "and"]  # Simply remove 'and'
    elif "after" in l4:
        ind = l4.index("after")
        l5 = l4[ind + 1:] + l4[:ind]  # Reorder actions as per 'after'
    else:
        l5 = l4

    # Final output: Flatten the output into one single list
    final_output = []
    final_output.extend(l5)

    # Return the fully resolved command as a flat list
    return final_output

# Example usage:
command = "jump thrice and turn left"
output = causal_model_bu(command)
print(output)

['JUMP', 'JUMP', 'JUMP', 'LTURN']


In [1]:
from datasets import load_dataset

In [2]:
scan_simple = load_dataset('scan', 'simple', trust_remote_code=True)

In [6]:
scan_simple['train'][:10]

{'commands': ['jump opposite right twice and turn opposite right thrice',
  'run opposite left after walk right',
  'walk after run around right twice',
  'look around right thrice and turn left',
  'walk opposite left twice and walk opposite right thrice',
  'look right and turn opposite right twice',
  'walk opposite left thrice after walk twice',
  'run thrice after jump around left',
  'run opposite left after walk opposite right',
  'run around left twice and run around right'],
 'actions': ['I_TURN_RIGHT I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT',
  'I_TURN_RIGHT I_WALK I_TURN_LEFT I_TURN_LEFT I_RUN',
  'I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_WALK',
  'I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RI

In [1]:
a = []
a[0]

IndexError: list index out of range

In [5]:
a = [[1,2],3,[4,5]]
b = []
b.extend(a[2:])
b.extend(a[:1])
b

[[4, 5], [1, 2]]

In [7]:
from itertools import chain

A = [[1,[2,5]], [3,4]]

list(chain(*A))

[1, [2, 5], 3, 4]